In [ ]:
import pandas as pd
import numpy as np
import import_ipynb
import anvil.server

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
from feeds import BackFeed,LiveFeed,DataFeed,clean_feed
from featfuncs import add_addl_features_feed,add_sym_feature_feed

In [ ]:
from feed_env import FeedEnv, Episode

In [ ]:
from ipynb.fs.defs.scanner import compute_gaps

In [ ]:
class Backtest():
    def __init__(self,feed,tickers=None,add_features=True,
                 target=.05,stop=.02,txcost=.001,remote=False,
                 data_cols=None,rpcname=None,loc_exit=True,rem_exit=False,
                 scan=True,topk=5):
        if add_features:
            add_addl_features_feed(feed,tickers=tickers,drop_ta=False)
            add_sym_feature_feed(feed,tickers)
        self.feed=feed
        self.feed.init_counters()
        self.results={}
        self.returns={}
        self.total=0
        self.target=target
        self.stop=stop
        self.txcost=txcost
        self.remote=remote
        self.loc_exit=loc_exit
        self.rem_exit=rem_exit
        self.data_cols=data_cols
        if self.remote: 
            anvil.server.connect("STKYCIXUMGFV2CURGMCMOFUV-MBK5ZXFC4MCKYKOE")
            self.model_type='rpc'
        else: self.model_type='none'
        self.rpcname=rpcname
        self.scan=scan
        self.topk=topk
        if self.scan: 
            self.tickersD=compute_gaps(self.feed,self.topk)
            self.gaptickers=[list(self.tickersD[d]['ticker'].values) for d in self.tickersD]
            self.gaptickers=list(set(sum(self.gaptickers,[])))
    def run_all(self,tickers=None,model=None):
        if self.scan:
            for t in self.gaptickers: 
                clean_feed(self.feed,t)
                self.results[t]={}
                self.returns[t]=0
            for date in tqdm(self.tickersD):
                for ticker in tqdm(self.tickersD[date]['ticker'].values):
                    self.run(ticker,model=model,date=date)
        else:
            for ticker in tqdm(tickers):
                clean_feed(self.feed,ticker)
                self.results[ticker]={}
                self.returns[ticker]=0
                self.run(ticker,model=model)
    def run(self,ticker,model=None,date=None):
        env=FeedEnv(self.feed,ticker=ticker,
                    target=self.target,stoploss=self.stop,
                    txcost=self.txcost)
        self.env=env
        env.set_state_type('dict')
        if self.remote==True:
            env.set_state_cols(self.data_cols)
            episode=Episode(env,model_type=self.model_type,remote=True,
                            rpcname=self.rpcname,rem_exit=self.rem_exit)
        else:
            env.set_state_cols(model.data_cols)
            episode=Episode(env,policy=model,loc_exit=self.loc_exit)
        env.set_episode(episode)
        if self.scan: dates=[date]
        else: dates=tqdm(self.feed.data[ticker].Date.unique()[1:])
        for date in dates:
            env.set_date(date=date)
            self.feed.init_counters(date)
            tot,rews,acts=episode.run_episode()
            self.results[ticker][date]={'tot':tot,'rew':rews,'acts':acts}
            self.returns[ticker]+=tot
            self.total+=tot
            env.reset()
            # print(tot,env.date)